## Preprocessing

In [1]:
#load data
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("nsmc")

train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])


Using custom data configuration default
Reusing dataset nsmc (C:\Users\ford0\.cache\huggingface\datasets\nsmc\default\1.1.0\bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


In [2]:
#simplify

import re

docs = dataset['train']['document'] + dataset['test']['document']
label = dataset['train']['label'] + dataset['test']['label']

processed_docs = [re.sub("[\s]+", " ", re.sub("[^가-힣a-zA-Z0-9]", " ", doc)) for doc in docs]

In [3]:
processed_docs[20:50]

['나름 심오한 뜻도 있는 듯 그냥 학생이 선생과 놀아나는 영화는 절대 아님',
 '보면서 웃지 않는 건 불가능하다',
 '재미없다 지루하고 같은 음식 영화인데도 바베트의 만찬하고 넘 차이남 바베트의 만찬은 이야기도 있고 음식 보는재미도 있는데 이건 볼게없다 음식도 별로 안나오고 핀란드 풍경이라도 구경할랫는데 그것도 별로 안나옴 ',
 '절대 평범한 영화가 아닌 수작이라는걸 말씀드립니다 ',
 '주제는 좋은데 중반부터 지루하다',
 '다 짤랐을꺼야 그래서 납득할 수 없었던거야 그럴꺼야 꼭 그랬던걸꺼야 ',
 'kl2g 고추를 털어버려야 할텐데',
 '카밀라벨 발연기',
 '재밋는뎅',
 '센스있는 연출력 탁월한 캐스팅 90년대의 향수 그래서 9점 ',
 '엄포스의 위력을 다시 한번 깨닫게 해준 적 남 꽃검사님도 연기 정말 좋았어요 완전 명품드라마 ',
 '졸쓰레기 진부하고말도안됌 아 시간아까워',
 '재밌는데 별점이 왜이리 낮은고',
 '1 라도 기대했던 내가 죄인입니다 죄인입니다 ',
 '아직도 이 드라마는 내인생의 최고 ',
 '패션에 대한 열정 안나 윈투어 ',
 '키이라 나이틀리가 연기하고자 했던건 대체 정신장애일까 틱장애일까',
 '허허 원작가 정신나간 유령이라 재미있겠네요 ',
 '포스터는 있어보이는데 관객은 114명이네',
 '이 영화가 왜 이렇게 저평가 받는지 모르겠다',
 '단순하면서 은은한 매력의 영화',
 ' 다 알바생인가 내용도 없고 무서운거도 없고 웃긴거도 하나도 없음 완전 별싱거운 영화 내 시간 넘 아까움 완전 낚임',
 '오게두어라 서리한이 굶주렸다 ',
 '정말 맘에 들어요 그래서 또 보고싶은데 또 보는 방법이 없네 ',
 '윤제문이라는 멋진 배우를 발견하게 됐어요 소소한 일탈이 잔잔한 미소를 머금게 합니다 음악은 조금 아쉽네요 8점 주고 싶은데 평점 올리고 싶어 10점 줄게요 ',
 '평점에속지마시길시간낭비 돈낭비임',
 '리얼리티가 뛰어나긴 한데 큰 공감은 안간다 이민기캐릭터는 정신의학상 분노조절장애 초기 증상일거다 툭하면 사람

In [30]:
from konlpy.tag import Okt

def tokenize(docs):

  okt = Okt()

  for doc in docs:
    tokenlist = okt.pos(doc)
    temp = []
    for w in tokenlist:
      if w[1] in ['Noun', 'Verb', 'Adjective', 'Adverb', 'Exclamation', 'Foreign', 'Alpha', 'Number', 'Unknown']:
        temp.append(w[0])
    res.append(temp)
  return res


In [4]:
# tokenize

# from konlpy.tag import Mecab

# mecab = Mecab()

# res = []

# for doc in docs:
#   tokenlist = mecab.pos(doc)
#   for w in tokenlist:
#     if w[1] in ['NNG', 'NNP', 'NNB', 'NNBC', 'NP' 'VV', 'VA', 'IC', 'SN', 'SL', 'MAG', 'MAJ']:
#       res.append(w[1])

from konlpy.tag import Okt
import pickle

okt = Okt()

res = []

if not os.path.isfile('senspos.pickle'):
  for doc in docs:
    tokenlist = okt.pos(doc)
    temp = []
    for w in tokenlist:
      if w[1] in ['Noun', 'Verb', 'Adjective', 'Adverb', 'Exclamation', 'Foreign', 'Alpha', 'Number', 'Unknown']:
        temp.append(w[0])
    res.append(temp)

  with open("senspos.pickle", 'wb') as f:
    pickle.dump(res, f)
  
else:
  with open("senspos.pickle", 'rb') as f:
    res = pickle.load(f)
  


In [5]:
# padding
maxlen = max(len(x) for x in res)
padded_sens = []
for i in range(len(res)):
  sen = res[i]
  temp = sen + [" <PAD/>"] * (maxlen - len(sen))
  padded_sens.append(temp)


In [6]:
# vocab to index

import nltk
tokens = [t for d in padded_sens for t in d]
text = nltk.Text(tokens, name='NSMC')
word_count = text.vocab()
vocabulary_inv = [x[0] for x in word_count.most_common()]
vocabulary = {x:i for i, x in enumerate(vocabulary_inv)}


In [7]:


vocabulary_inv[:10]

['<PAD/>', '영화', '너무', '정말', '진짜', '이', '점', '연기', '평점', '것']

In [ ]:
vocabulary.items()


In [22]:
# word 2 vec
from gensim.models import word2vec
import numpy as np

w2v = word2vec.Word2Vec.load("../data/w2v/ko.bin")

w2v = {w: w2v[w] if w in w2v else np.random.uniform(-0.25, 0.25, w2v.vector_size) for w in vocabulary_inv}


C:\Users\ford0\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
C:\Users\ford0\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [23]:
w2v['안녕']

array([-4.48758937e-02, -5.26038110e-01,  2.18058324e+00, -2.70098805e-01,
       -1.22612631e+00, -4.35174644e-01,  8.78038049e-01,  1.78903091e+00,
        5.13344407e-01,  8.00866067e-01,  1.33781850e-01,  4.23676372e-01,
        6.25718057e-01, -1.49838462e-01, -2.65716583e-01, -1.18363452e+00,
        1.09828494e-01,  6.56335890e-01,  9.90437388e-01, -2.88908720e-01,
       -4.82253104e-01, -1.71825096e-01, -5.22048175e-01, -1.37081638e-01,
        4.01715070e-01, -3.54384005e-01, -4.38563734e-01,  3.37939188e-02,
       -4.21380460e-01, -2.11324722e-01,  5.34786046e-01,  5.26668072e-01,
       -3.78847629e-01, -1.59311071e-01,  1.77993524e+00, -1.28354877e-01,
        4.53990996e-01,  6.38647795e-01,  3.22042465e-01, -6.49676397e-02,
       -9.30086493e-01,  6.84082747e-01, -1.41875729e-01, -1.49376774e+00,
        1.44657120e-03,  4.33810472e-01, -7.31874406e-02,  7.40463585e-02,
       -6.44519210e-01,  1.11461449e+00, -8.20367783e-02, -5.76601565e-01,
        1.71026981e+00,  

In [ ]:
embedding_dim = 200
filter_sizes = (3, 4, 5)
num_filters = 100
dropout = 0.5
hidden_dims = 100

batch_size = 50
num_epochs = 10
min_word_count = 1
context = 10

In [31]:
# X_train = tokenize(train_df['document'])
X_train = res[:len(train_df)]
y_train = train_df['label']
# X_test = tokenize(test_df['document'])
X_test = res[len(train_df):]
y_test = test_df['label']

X_train = np.stack([np.stack([w2v[w] for w in sen]) for sen in X_train])
X_test = np.stack([np.stack([w2v[w] for w in sen]) for sen in X_test])

In [28]:
X_train

In [ ]:
from tensorflow import keras

m = keras.layers.Input(shape=(X_test.shape[1] ,embedding_dim))

# build conv block
conv_blocks = []
for s in filter_sizes:
  conv = keras.layers.Conv1D(filters=num_filters,
                         kernel_size=s,
                         padding="valid",
                         activation="relu",
                         strides=1)(z)
  conv = keras.layers.MaxPooling1D(pool_size=2)(conv)
  conv = keras.layers.Flatten()(conv)
  conv_blocks.append(conv)
m = keras.layers.Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]

m = keras.layers.Dropout(dropout)(m)
m = keras.layers.Dense(hidden_dims, activation="relu")(m)
model_output = keras.layers.Dense(1, activation="sigmoid")(m)

model = keras.Model(model_input, model_output)
model.compile(loss="binary_crossentropy", optimizer="adadelta", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=num_epochs,
          validation_data=(x_test, y_test), verbose=2)